There is a change in preprocessing in this notebook, we removed all the numbers from tweets, which helped in training more robust word2vec embeddings

Analysis on scraped dataset

### Import

In [0]:
import torch

In [0]:
import pandas as pd
import numpy as np

In [0]:
import pickle
import sys
import nltk
from nltk.stem.porter import *

from sklearn.feature_extraction.text import TfidfVectorizer
import matplotlib.pyplot as plt
import seaborn
%matplotlib inline

In [0]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


In [0]:
cf_data_1 = pd.read_csv('/content/drive/My Drive/Hate_Speech_Detection_git/data_1/hatespeech_NAACL_SRW.csv',encoding = "ISO-8859-1")
cf_data_2 = pd.read_csv('/content/drive/My Drive/Hate_Speech_Detection_git/data_1/hatespeech_NLP+CSS.csv')

cf_data_3 = pd.read_csv('/content/drive/My Drive/Hate_Speech_Detection_git/data_2/labeled_data.csv',encoding = "ISO-8859-1")
## this is the scraped data

cf_data_3.rename({'Unnamed: 0':'ID','tweet':'Tweets'},axis=1,inplace=True)

labels_1 = pd.read_csv('/content/drive/My Drive/Hate_Speech_Detection_git/data_1/NAACL_SRW_2016.csv',header=None,names=['ID','class'])
labels_2 = pd.read_csv('/content/drive/My Drive/Hate_Speech_Detection_git/data_1/NLP+CSS_2016.csv',sep='\s')

labels_2.rename({'TweetID':'ID','Expert':'class'},axis=1,inplace=True)

cf_data_1.rename({'Unnamed: 0':'index_col'},axis=1,inplace=True)
cf_data_2.rename({'Unnamed: 0':'index_col'},axis=1,inplace=True)

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:10: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support regex separators (separators > 1 char and different from '\s+' are interpreted as regex); you can avoid this warning by specifying engine='python'.
  # Remove the CWD from sys.path while we load stuff.


In [0]:
real_world_test = pd.read_csv('/content/drive/My Drive/Hate_Speech_Detection_git/real_world_test/realDonaldTrump_tweets_extended.csv')

In [0]:
real_world_test_racist = pd.read_csv('/content/drive/My Drive/Hate_Speech_Detection_git/real_world_test/racist_scarped_extended.csv', names=['timestamp','text'])

In [0]:
pok = pd.read_csv('/content/drive/My Drive/Hate_Speech_Detection_git/real_world_test/pok.csv', names=['timestamp','text'])

In [0]:
pok = pok.drop_duplicates(subset='text')

### Function for merging

In [0]:
def label_merging(data, labels):
    labels['ID'] = labels['ID'].astype(int)
    print(labels['ID'].nunique())
    print('Null IDs in data 1 = ' ,data['ID'].isna().sum())
    
    data['ID'].fillna(0,inplace=True)
    data['ID'] = data['ID'].astype(int)
    
    print('data shape ='  ,data.shape)
    print('IDs common in data and labels =',sum(data['ID'].isin(labels['ID'])))
    
    train = data.merge(labels, on='ID',how='inner')#['class'].isna().sum()
    return train

In [0]:
train_1 = label_merging(cf_data_1,labels_1)

train_2 = label_merging(cf_data_2, labels_2)

train_3 = cf_data_3.copy()

16849
Null IDs in data 1 =  2
data shape = (16037, 11)
IDs common in data and labels = 11238
6909
Null IDs in data 1 =  0
data shape = (6271, 11)
IDs common in data and labels = 6271


In [0]:
t1 = train_1[['ID','Tweets','class']]
t2 = train_2[['ID','Tweets','class']]
t3 = train_3[['ID','Tweets','class']]
merged = pd.concat([t1,t2,t3],axis=0).reset_index(drop=True)

### Basic Preprocessing

In [0]:
train = merged.copy()

train.rename(columns={'Tweets':'tweet'},inplace=True)
train['tweet'] = train['tweet'].astype(str)

In [0]:
# train = train.drop_duplicates(subset='ID')
# train.to_csv('train.csv',index=False, header=True)

In [0]:
def clean_remove_b(data):  
  data.rename(columns={'text':'tweet'},inplace=True)
  data['tweet'] = data['tweet'].astype(str)

  data['tweet'] = data['tweet'].apply(lambda x:x[2:] if x[0:2]=="b'" or 'b"' else x)

In [0]:
clean_remove_b(real_world_test)
clean_remove_b(real_world_test_racist)
clean_remove_b(pok)

Every word followed by @ is some twitter ID of an user, which shouldn't be considered in our analysis, so lets do the stemming, where we remove @ alonwith the word followed by it

In [0]:
def preprocess(data):
  data['tweet'] = data['tweet'].apply(lambda x:' '.join(i for i in [a for a in x.split() if a.find('@')==-1]))
  data['tweet'] = data['tweet'].apply(lambda x:' '.join(i for i in [a for a in x.split() if a.find('http')==-1]))
  
  ## we are removing hashtags now, but while doing transfer learning, to learn the embeddings we didnt remove these, 
  ## just to include such words in our vocabulary
  
  data['tweet'] = data['tweet'].apply(lambda x:' '.join(i for i in [a for a in x.split() if a.find('#')==-1]))
  data['tweet'] = data['tweet'].apply(lambda x:''.join([i for i in x if not i.isdigit()]))
  data['tweet'] = data['tweet'].apply(lambda x: " ".join(x.lower() for x in x.split()))
  data['tweet'] = data['tweet'].str.replace('[^\w\s]','')

  # import nltk
  # nltk.download('stopwords')

  # from nltk.corpus import stopwords
  # stop = stopwords.words('english')
  # data['tweet'] = data['tweet'].apply(lambda x: " ".join(x for x in x.split() if x not in stop))

  # remove_word = ['rt','mkr','im']
  # data['tweet'] = data['tweet'].apply(lambda x: " ".join(x for x in x.split() if x not in remove_word))

In [0]:
preprocess(train)
preprocess(real_world_test)
preprocess(real_world_test_racist)
preprocess(pok)

Doesnt really make sense to remove rare words, i.e. the words with count 1. Because we might lose hateful words this way

In [0]:
def preprocess_2(data):
  from textblob import TextBlob
  nltk.download('punkt')

  from textblob import Word
  nltk.download('wordnet')
  data['tweet'] = data['tweet'].apply(lambda x: " ".join([Word(word).lemmatize() for word in x.split()]))
  data['tweet'].head()

In [0]:
# preprocess_2(train)
# preprocess_2(real_world_test)
# preprocess_2(real_world_test_racist)
# preprocess_2(pok)

# for bert we wont lemmatize it

In [0]:
# real_world_test = real_world_test[real_world_test['tweet'].apply(lambda x:len(x)>1)]

### Target creation

In [0]:
train['class'].unique()#.isna().sum()

array(['racism', 'none', 'sexism', 'neither', 'both', 2, 1, 0],
      dtype=object)

In [0]:
train['class'].replace(['racism', 'sexism',0, 1, 'both', 'none', 'neither',2],['hate','hate','hate','hate','hate','null','null','null'],inplace=True)
train['class'].value_counts()

hate    24942
null    17422
Name: class, dtype: int64

In [0]:
train['class'].replace(['null','hate'],[0,1],inplace=True)

### dropping the duplicate

In [0]:
# hate_text = train[train['class']==1]['tweet']
# null_text = train[train['class']==0]['tweet']

In [0]:
# hate_text.to_csv(r'hate_speech.txt', header=None, index=None, sep=' ')

In [0]:
# null_text.to_csv(r'null_speech.txt', header=None, index=None, sep=' ')

In [0]:
sum(train['ID'].value_counts()>1)

2760

In [0]:
train.shape

(42364, 3)

In [0]:
train = train.drop_duplicates(subset='ID')

real_world_test = real_world_test.drop_duplicates(subset='id')

### Out of time #1 -  Trump Tweets

In [0]:
real_world_test_original = pd.read_csv('/content/drive/My Drive/Hate_Speech_Detection_git/real_world_test/realDonaldTrump_tweets_extended.csv')

check_df = real_world_test_original.iloc[400:500]
check_df['processed_tweet'] = real_world_test.iloc[400:500]['tweet']
check_df['preds'] = xgb_w2v.predict(out_of_time_test)
check_df['hate_probab'] = xgb_w2v.predict_proba(out_of_time_test)[::,-1]

# false_index = check_df[check_df['class']==xgb_w2v.predict(X_test_word_average)]['ID'].unique()

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  after removing the cwd from sys.path.
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: h

In [0]:
check_df[check_df['hate_probab']>0.5]

,id,created_at,text,processed_tweet,preds,hate_probab
421,1182290380542185472,2019-10-10 13:42:47,b'Why isn\xe2\x80\x99t the IG investigating his so-called Whistleblower? All bad info!',isnxexxt ig investigating socalled whistleblower bad info,1,0.536151
444,1181940394604535809,2019-10-09 14:32:04,"b'....DACA stand, with all of its negative legal implications, the Republicans and Democrats will have a DEAL to let them stay in our Country, in very short order. It would actually benefit DACA, and be done the right way!'",daca stand negative legal implication republican democrat deal let stay country short order would actually benefit daca done right way,1,0.565205
445,1181940389990805504,2019-10-09 14:32:03,"b'President Obama said that he did not have the right to sign DACA, that it will never hold up in court. He signed it anyway! If the Supreme Court upholds DACA, it gives the President extraordinary powers, far greater than ever thought. If they do what is right and do not let......'",president obama said right sign daca never hold court signed anyway supreme court upholds daca give president extraordinary power far greater ever thought right let,1,0.564453
447,1181935014709202947,2019-10-09 14:10:41,b'Crooked Hillary should try it again! https://t.co/UjfIpZp1FA',crooked hillary try,1,0.802087
450,1181933850991235073,2019-10-09 14:06:04,"b'Why doesn\xe2\x80\x99t the ICIG do something about this Scam? He should have never let it start, a Dem Hoax! https://t.co/UCbObppWbA'",doesnxexxt icig something scam never let start dem hoax,1,0.761173
453,1181932207767789571,2019-10-09 13:59:32,"b'So true Don, and this was a big lie. Look what happened to Friends of Trump, for less. Can\xe2\x80\x99t let this pass! https://t.co/u2To514DNq'",true big lie look happened friend trump le canxexxt let pas,1,0.648571
459,1181929745518403585,2019-10-09 13:49:45,"b'Great support from GOP in fighting the Radical Left, Do Nothing Democrats! https://t.co/qjskHYkQpD'",great support gop fighting radical left nothing democrat,1,0.668431
469,1181891356274413570,2019-10-09 11:17:12,"b'So make all Member Countries pay, not just the United States! https://t.co/IVbE4MqBVl'",make member country pay united state,1,0.520830
474,1181883934776401920,2019-10-09 10:47:43,"b'A Total Scam by the Do Nothing Democrats. For the good of the Country, this Wirch Hunt should end now! https://t.co/G2nPapozye'",total scam nothing democrat good country wirch hunt end,1,0.514024
482,1181750932444520448,2019-10-09 01:59:12,"b'RT @senatemajldr: The USMCA would create 176,000 jobs for American workers and generate $68 billion in wealth for America. House Democrats\xe2\x80\xa6'",usmca would create job american worker generate billion wealth america house democratsxexxa,1,0.552511


### Out of Time #2 -  Racist Tweets
P.S. Not all of them are racist actually, we need to check it manually

In [0]:
real_world_test_original_racist = pd.read_csv('/content/drive/My Drive/Hate_Speech_Detection_git/real_world_test/racist_scarped_extended.csv', names=['timestamp','text'])

check_df_2 = real_world_test_original_racist.iloc[200:400]
check_df_2['processed_tweet'] = real_world_test_racist.iloc[200:400]['tweet']
check_df_2['preds'] = xgb_w2v.predict(out_of_time_test_racist)
check_df_2['hate_probab'] = xgb_w2v.predict_proba(out_of_time_test_racist)[::,-1]

# false_index = check_df[check_df['class']==xgb_w2v.predict(X_test_word_average)]['ID'].unique()

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  after removing the cwd from sys.path.
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: h

In [0]:
check_df_2[check_df_2['hate_probab']>0.4]

,timestamp,text,processed_tweet,preds,hate_probab
207,2019-10-24 09:27:26,b'\xe2\x81\xa6@TheJire\xe2\x81\xa9 and \xe2\x81\xa6@neccowafers\xe2\x81\xa9 harass gay black men with homophobic and racist slurs #gaymers #blacklgbt #black #racist https://t.co/8oItTxtyFH',harass gay black men homophobic racist slur,1,0.926347
209,2019-10-24 09:25:12,"b'@MiriamElman @DeborahProudJew @SkepticReview89 @transform6789 @DavidHirsh @bariweiss @fathomjournal AntiZionism = antiSemitism\n\nThe resistance to a Jewish State is as old as the idea itself, and it\xe2\x80\x99s not based on \xe2\x80\x98Israel\xe2\x80\x99s behaviour\xe2\x80\x99, as some on the Left like to portray it. (See above) The resistance to Israel is #racist.'",antizionism antisemitismnnthe resistance jewish state old idea itxexxs based xexxisraelxexxs behaviourxexx left like portray see resistance israel,0,0.474567
210,2019-10-24 09:21:40,b'I really doubt in this whole world that white person of anywhere can ever get over that stupid fact of thinking they know a Shit and are shit ... when truly they are this worlds biggest Fools Nd Bullshit ... #WhitePrivilege #WhiteSupremacist #Cracker #racist https://t.co/U2SK4X3Bnx',really doubt whole world white person anywhere ever get stupid fact thinking know shit shit truly world biggest fool nd bullshit,0,0.480192
216,2019-10-24 08:46:55,"b'RT @ClimateCritic18: #auspol\n\nYou are labeled #racist, privileged, white supremacist, #bigoted, #ableist, #ignorant and a nazi just for the\xe2\x80\xa6'",labeled privileged white supremacist nazi thexexxa,1,0.688225
217,2019-10-24 08:43:43,"b'RT @ClimateCritic18: #auspol\n\nYou are labeled #racist, privileged, white supremacist, #bigoted, #ableist, #ignorant and a nazi just for the\xe2\x80\xa6'",labeled privileged white supremacist nazi thexexxa,1,0.688225
...,...,...,...,...,...
358,2019-10-24 02:35:19,b'@cgrapski @AfricaMustWake @Nastyness61 #racist prosecutors employed by a predominantly white judiciary with #ImpeachedCops and equally racist and unqualified judges like #JudgeDonHLester @flcourts who subvert equal rights and protection under color of law #MandatedPOLY',prosecutor employed predominantly white judiciary equally racist unqualified judge like subvert equal right protection color law,1,0.732867
370,2019-10-24 02:02:40,"b""Question.... Isn't not letting someone say nigger based on their skin color kinda racist in itself. It's a paradox. #racist #nword""",question isnt letting someone say nigger based skin color kinda racist paradox,1,0.601461
371,2019-10-24 02:02:30,"b'@CornelWest @RefuseFascism YOU are the #Fascist and 100% ignorant\n\nIt was the founders of the #KKK (#democrats) doing the lynching \n\nAlso, #lynching\'s happened to white men by the 1000s, you dont own the term ""lynching""\n\nDemocrats are the #racist\n\nExhibit A: #BLACKFACE https://t.co/NR0awdyGr7'",ignorantnnit founder lynching nnalso happened white men dont term lynchingnndemocrats,1,0.761431
379,2019-10-24 01:57:39,"b'@Ilhan Thank you for showing your true colors. You try to divide and destroy our country, make racist comments, put us down, insult us, lie about us, lie about our President. #WalkAway #racist #Hypocrite #DemocratsAreCorrupt #IStandWithPresidentTrump'",thank showing true color try divide destroy country make racist comment put u insult u lie u lie president,1,0.795588


### Out of time 3 - POK

In [0]:
pok_original = pd.read_csv('/content/drive/My Drive/Hate_Speech_Detection_git/real_world_test/pok.csv', names=['timestamp','text'])
pok_original = pok_original.drop_duplicates(subset='text')

check_df_3 = pok_original.iloc[200:400]
check_df_3['processed_tweet'] = pok.iloc[200:400]['tweet']
check_df_3['preds'] = xgb_w2v.predict(pok_out_of_time)
check_df_3['hate_probab'] = xgb_w2v.predict_proba(pok_out_of_time)[::,-1]

# false_index = check_df[check_df['class']==xgb_w2v.predict(X_test_word_average)]['ID'].unique()

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/s

In [0]:
check_df_3[check_df_3['hate_probab']>0.4]

,timestamp,text,processed_tweet,preds,hate_probab
405,2019-10-23 15:58:58,b'@rehankayanii Whole world knows #Who_is_Modi And What he can do. Now Kashmir is gone from your hands Now its time to take our #Pok Back.',whole world know kashmir gone hand time take back,0,0.471642
472,2019-10-23 14:20:20,"b'RT @Je_suis_perdu_7: @Rabipirzada There is nothing like IOK but yes #POK exists where coward #PakistanArmy kidnap, kill and rape people.\n\n#\xe2\x80\xa6'",nothing like iok yes exists coward kidnap kill rape,1,0.819460
490,2019-10-23 13:59:36,b'As per the libtards only #Muslims are humans and only they have #HumanRights all others are kafirs hence rights dont matter.. \n#PoK \n#attackonfreedom',per libtards human others kafir hence right dont matter,0,0.411190
492,2019-10-23 13:58:30,b'Dear @Ilhan they have been asked to prove their citizenship because they are living illegally in india.... you are too blind to see what is happing in #POK and #Baluchistan \n\nDid you ask this question to yourself why all the terrorists attack has been done by Muslims ??? https://t.co/BFKR7G608d',dear asked prove citizenship living illegally india blind see happing nndid ask question terrorist attack done muslim,1,0.612894
510,2019-10-23 13:44:26,"b'RT @Drg_nn: @DextrousNinja https://t.co/5CV3cfS9Of\nSomali Muslim Rep #IlhanOmar has lied, cheated &amp; played sympathy card to get her US citi\xe2\x80\xa6'",muslim rep lied cheated amp played sympathy card get u citixexxa,1,0.683725
552,2019-10-23 13:19:50,b'RT @BasudevOfficial: #ImranKhan declared #Emergence at #POK by scared of there sound gathered for freedom from #Pakistan .\n\nBy showing this\xe2\x80\xa6',declared scared sound gathered freedom nnby showing thisxexxa,0,0.465755
585,2019-10-23 12:47:25,"b'@Imamofpeace @IlhanMN What she have to say on butchering of #Kurds and Muslims in Middle East by Islamic fundamentalists. She ignore ethnic cleansing of #ahmadias, rape &amp; conversion of minority girls by #pakistani Muslims in #blochistan &amp; #POK . She should be ashamed of herself. We don\xe2\x80\x99t give a damn !'",say butchering muslim middle east islamic fundamentalist ignore ethnic cleansing rape amp conversion minority girl muslim amp ashamed donxexxt give damn,1,0.913208
587,2019-10-23 12:45:39,b'#IndianArmedForces must liberate #PoK tonight and give freedom to kashmiris living there from the atrocities of terrorist #PakistanArmy . 26 October 1947 is blackday for the people living in #PoK. #PoK is under illegal occupation of #PakistanArmy. #loc',must liberate tonight give freedom kashmiri living atrocity terrorist october blackday people living illegal occupation,1,0.750307
621,2019-10-23 12:05:20,"b""@arifaajakia Holy crap.. That's a United Nations car.. It is the @UN delegation.... People of #POK are protesting in front of them asking #pakistan to leave #kashmir. Hahaha \xf0\x9f\xa4\xa3\xf0\x9f\x98\x82\xf0\x9f\x99\x88\xf0\x9f\x99\x89\xf0\x9f\x99\x8a #Shameonpakistan #Kashmir #Kashmiris""",holy crap thats united nation car delegation people protesting front asking leave hahaha xfxfxaxaxfxfxxxfxfxxxfxfxxxfxfxxa,0,0.446734
636,2019-10-23 11:46:21,b'@Ilhan Have ur say on killing in #PoK by #PakistanArmy since last 2days?\nY ur quite on that?',ur say killing since last daysny ur quite,1,0.642303


### BERT

In [0]:
from sklearn.model_selection import train_test_split
bert_train, bert_test = train_test_split(train, random_state=42, test_size=0.2)

In [0]:
!pip install pytorch-pretrained-bert


     |████████████████████████████████| 133kB 3.5MB/s 
     |████████████████████████████████| 645kB 11.0MB/s 


In [0]:
bert_train_input = pd.DataFrame({
    'id':range(len(bert_train)),
    'label':bert_train['class'],
    'alpha':['a']*bert_train.shape[0],
    'text': bert_train['tweet']
})


In [0]:
bert_test_input = pd.DataFrame({
    'id':range(len(bert_test)),
    'label':bert_test['class'],
    'alpha':['a']*bert_test.shape[0],
    'text': bert_test['tweet']
})


In [0]:
bert_train_input.to_csv('train.tsv', sep='\t', index=False, header=False)
bert_test_input.to_csv('dev.tsv', sep='\t', index=False, header=False)

In [0]:
!pip install torch
!pip install tqdm

In [0]:
from __future__ import absolute_import, division, print_function

import csv
import os
import sys
import logging

logger = logging.getLogger()
csv.field_size_limit(2147483647) # Increase CSV reader's field limit incase we have long text.


class InputExample(object):
    """A single training/test example for simple sequence classification."""

    def __init__(self, guid, text_a, text_b=None, label=None):
        """Constructs a InputExample.
        Args:
            guid: Unique id for the example.
            text_a: string. The untokenized text of the first sequence. For single
            sequence tasks, only this sequence must be specified.
            text_b: (Optional) string. The untokenized text of the second sequence.
            Only must be specified for sequence pair tasks.
            label: (Optional) string. The label of the example. This should be
            specified for train and dev examples, but not for test examples.
        """
        self.guid = guid
        self.text_a = text_a
        self.text_b = text_b
        self.label = label


class DataProcessor(object):
    """Base class for data converters for sequence classification data sets."""

    def get_train_examples(self, data_dir):
        """Gets a collection of `InputExample`s for the train set."""
        raise NotImplementedError()

    def get_dev_examples(self, data_dir):
        """Gets a collection of `InputExample`s for the dev set."""
        raise NotImplementedError()

    def get_labels(self):
        """Gets the list of labels for this data set."""
        raise NotImplementedError()

    @classmethod
    def _read_tsv(cls, input_file, quotechar=None):
        """Reads a tab separated value file."""
        with open(input_file, "r", encoding="utf-8") as f:
            reader = csv.reader(f, delimiter="\t", quotechar=quotechar)
            lines = []
            for line in reader:
                if sys.version_info[0] == 2:
                    line = list(unicode(cell, 'utf-8') for cell in line)
                lines.append(line)
            return lines


class BinaryClassificationProcessor(DataProcessor):
    """Processor for binary classification dataset."""

    def get_train_examples(self, data_dir):
        """See base class."""
        return self._create_examples(
            self._read_tsv(os.path.join(data_dir, "train.tsv")), "train")

    def get_dev_examples(self, data_dir):
        """See base class."""
        return self._create_examples(
            self._read_tsv(os.path.join(data_dir, "dev.tsv")), "dev")

    def get_labels(self):
        """See base class."""
        return ["0", "1"]

    def _create_examples(self, lines, set_type):
        """Creates examples for the training and dev sets."""
        examples = []
        for (i, line) in enumerate(lines):
            guid = "%s-%s" % (set_type, i)
            text_a = line[3]
            label = line[1]
            examples.append(
                InputExample(guid=guid, text_a=text_a, text_b=None, label=label))
        return examples

In [0]:
class InputFeatures(object):
    """A single set of features of data."""

    def __init__(self, input_ids, input_mask, segment_ids, label_id):
        self.input_ids = input_ids
        self.input_mask = input_mask
        self.segment_ids = segment_ids
        self.label_id = label_id


def _truncate_seq_pair(tokens_a, tokens_b, max_length):
    """Truncates a sequence pair in place to the maximum length."""

    # This is a simple heuristic which will always truncate the longer sequence
    # one token at a time. This makes more sense than truncating an equal percent
    # of tokens from each, since if one sequence is very short then each token
    # that's truncated likely contains more information than a longer sequence.
    while True:
        total_length = len(tokens_a) + len(tokens_b)
        if total_length <= max_length:
            break
        if len(tokens_a) > len(tokens_b):
            tokens_a.pop()
        else:
            tokens_b.pop()


def convert_example_to_feature(example_row):
    # return example_row
    example, label_map, max_seq_length, tokenizer, output_mode = example_row

    tokens_a = tokenizer.tokenize(example.text_a)

    tokens_b = None
    if example.text_b:
        tokens_b = tokenizer.tokenize(example.text_b)
        # Modifies `tokens_a` and `tokens_b` in place so that the total
        # length is less than the specified length.
        # Account for [CLS], [SEP], [SEP] with "- 3"
        _truncate_seq_pair(tokens_a, tokens_b, max_seq_length - 3)
    else:
        # Account for [CLS] and [SEP] with "- 2"
        if len(tokens_a) > max_seq_length - 2:
            tokens_a = tokens_a[:(max_seq_length - 2)]

    tokens = ["[CLS]"] + tokens_a + ["[SEP]"]
    segment_ids = [0] * len(tokens)

    if tokens_b:
        tokens += tokens_b + ["[SEP]"]
        segment_ids += [1] * (len(tokens_b) + 1)

    input_ids = tokenizer.convert_tokens_to_ids(tokens)

    # The mask has 1 for real tokens and 0 for padding tokens. Only real
    # tokens are attended to.
    input_mask = [1] * len(input_ids)

    # Zero-pad up to the sequence length.
    padding = [0] * (max_seq_length - len(input_ids))
    input_ids += padding
    input_mask += padding
    segment_ids += padding

    assert len(input_ids) == max_seq_length
    assert len(input_mask) == max_seq_length
    assert len(segment_ids) == max_seq_length

    if output_mode == "classification":
        label_id = label_map[example.label]
    elif output_mode == "regression":
        label_id = float(example.label)
    else:
        raise KeyError(output_mode)

    return InputFeatures(input_ids=input_ids,
                         input_mask=input_mask,
                         segment_ids=segment_ids,
                         label_id=label_id)

In [0]:
# %cd /content/drive/My Drive/Hate_Speech_Detection_git/Code_Notebooks
%cd /content

/content


In [0]:
import sys
# sys.path.append('/content/drive/My Drive/Hate_Speech_Detection_git/Code_Notebooks')
sys.path.append('/content')

In [0]:
# %load convert_examples_to_features.py

In [0]:
import torch
import pickle
from torch.utils.data import (DataLoader, RandomSampler, SequentialSampler, TensorDataset)
from torch.nn import CrossEntropyLoss, MSELoss

from tqdm import tqdm_notebook, trange
import os
from pytorch_pretrained_bert import BertTokenizer, BertModel, BertForMaskedLM, BertForSequenceClassification
from pytorch_pretrained_bert.optimization import BertAdam, WarmupLinearSchedule

from multiprocessing import Pool, cpu_count
# from tools import *
# from convert_examples_to_features import *

# OPTIONAL: if you want to have more information on what's happening, activate the logger as follows
import logging
logging.basicConfig(level=logging.INFO)

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [0]:
# The input data dir. Should contain the .tsv files (or other data files) for the task.
DATA_DIR = ""

# Bert pre-trained model selected in the list: bert-base-uncased, 
# bert-large-uncased, bert-base-cased, bert-large-cased, bert-base-multilingual-uncased,
# bert-base-multilingual-cased, bert-base-chinese.
BERT_MODEL = 'bert-base-cased'

# The name of the task to train.I'm going to name this 'yelp'.
TASK_NAME = 'hate_speech'

# The output directory where the fine-tuned model and checkpoints will be written.
OUTPUT_DIR = f'outputs/{TASK_NAME}/'

# The directory where the evaluation reports will be written to.
REPORTS_DIR = f'reports/{TASK_NAME}_evaluation_report/'

# This is where BERT will look for pre-trained models to load parameters from.
CACHE_DIR = 'cache/'

# The maximum total input sequence length after WordPiece tokenization.
# Sequences longer than this will be truncated, and sequences shorter than this will be padded.
MAX_SEQ_LENGTH = 128

TRAIN_BATCH_SIZE = 24
EVAL_BATCH_SIZE = 32
LEARNING_RATE = 2e-5

# NUM_TRAIN_EPOCHS = 1
NUM_TRAIN_EPOCHS = 3

RANDOM_SEED = 42
GRADIENT_ACCUMULATION_STEPS = 1
WARMUP_PROPORTION = 0.1
OUTPUT_MODE = 'classification'

CONFIG_NAME = "config.json"
WEIGHTS_NAME = "pytorch_model.bin"

In [0]:
!rm -rf outputs
!rm -rf reports

In [0]:
output_mode = OUTPUT_MODE

cache_dir = CACHE_DIR

if os.path.exists(REPORTS_DIR) and os.listdir(REPORTS_DIR):
        REPORTS_DIR += f'/report_{len(os.listdir(REPORTS_DIR))}'
        os.makedirs(REPORTS_DIR)
if not os.path.exists(REPORTS_DIR):
    os.makedirs(REPORTS_DIR)
    REPORTS_DIR += f'/report_{len(os.listdir(REPORTS_DIR))}'
    os.makedirs(REPORTS_DIR)

if os.path.exists(OUTPUT_DIR) and os.listdir(OUTPUT_DIR):
        raise ValueError("Output directory ({}) already exists and is not empty.".format(OUTPUT_DIR))
if not os.path.exists(OUTPUT_DIR):
    os.makedirs(OUTPUT_DIR)

In [0]:
%%time
processor = BinaryClassificationProcessor()
train_examples = processor.get_train_examples(DATA_DIR)
train_examples_len = len(train_examples)

label_list = processor.get_labels() # [0, 1] for binary classification
num_labels = len(label_list)

num_train_optimization_steps = int(train_examples_len / TRAIN_BATCH_SIZE / GRADIENT_ACCUMULATION_STEPS) * NUM_TRAIN_EPOCHS

# Load pre-trained model tokenizer (vocabulary)
tokenizer = BertTokenizer.from_pretrained('bert-base-cased', do_lower_case=False)

label_map = {label: i for i, label in enumerate(label_list)}
train_examples_for_processing = [(example, label_map, MAX_SEQ_LENGTH, tokenizer, OUTPUT_MODE) for example in train_examples]

INFO:pytorch_pretrained_bert.file_utils:https://s3.amazonaws.com/models.huggingface.co/bert/bert-base-cased-vocab.txt not found in cache, downloading to /tmp/tmp4f5wa3z6
100%|██████████| 213450/213450 [00:00<00:00, 2464303.83B/s]
INFO:pytorch_pretrained_bert.file_utils:copying /tmp/tmp4f5wa3z6 to cache at /root/.pytorch_pretrained_bert/5e8a2b4893d13790ed4150ca1906be5f7a03d6c4ddf62296c383f6db42814db2.e13dbb970cb325137104fb2e5f36fe865f27746c6b526f6352861b1980eb80b1
INFO:pytorch_pretrained_bert.file_utils:creating metadata file for /root/.pytorch_pretrained_bert/5e8a2b4893d13790ed4150ca1906be5f7a03d6c4ddf62296c383f6db42814db2.e13dbb970cb325137104fb2e5f36fe865f27746c6b526f6352861b1980eb80b1
INFO:pytorch_pretrained_bert.file_utils:removing temp file /tmp/tmp4f5wa3z6
INFO:pytorch_pretrained_bert.tokenization:loading vocabulary file https://s3.amazonaws.com/models.huggingface.co/bert/bert-base-cased-vocab.txt from cache at /root/.pytorch_pretrained_bert/5e8a2b4893d13790ed4150ca1906be5f7a03d6c

CPU times: user 256 ms, sys: 23 ms, total: 279 ms
Wall time: 660 ms


In [0]:
%%time
process_count = cpu_count() - 1
if __name__ ==  '__main__':
    print(f'Preparing to convert {train_examples_len} examples..')
    print(f'Spawning {process_count} processes..')
    with Pool(process_count) as p:
        train_features = list(tqdm_notebook(p.imap(convert_example_to_feature, train_examples_for_processing), total=train_examples_len))
        # here we are importing these processing functions directly from the notebook, so directly using it, instead of module.function type

Preparing to convert 31604 examples..
Spawning 1 processes..



CPU times: user 9min 2s, sys: 17.6 s, total: 9min 20s
Wall time: 12min 32s


In [0]:
!rm train_features.pkl

rm: cannot remove 'train_features.pkl': No such file or directory


In [0]:
with open(DATA_DIR + "train_features.pkl", "wb") as f:
    pickle.dump(train_features, f)

In [0]:
# Load pre-trained model (weights)
model = BertForSequenceClassification.from_pretrained(BERT_MODEL, cache_dir=CACHE_DIR, num_labels=num_labels)

INFO:pytorch_pretrained_bert.file_utils:https://s3.amazonaws.com/models.huggingface.co/bert/bert-base-cased.tar.gz not found in cache, downloading to /tmp/tmpgucbs7jt
100%|██████████| 404400730/404400730 [00:08<00:00, 49788078.16B/s]
INFO:pytorch_pretrained_bert.file_utils:copying /tmp/tmpgucbs7jt to cache at cache/a803ce83ca27fecf74c355673c434e51c265fb8a3e0e57ac62a80e38ba98d384.681017f415dfb33ec8d0e04fe51a619f3f01532ecea04edbfd48c5d160550d9c
INFO:pytorch_pretrained_bert.file_utils:creating metadata file for cache/a803ce83ca27fecf74c355673c434e51c265fb8a3e0e57ac62a80e38ba98d384.681017f415dfb33ec8d0e04fe51a619f3f01532ecea04edbfd48c5d160550d9c
INFO:pytorch_pretrained_bert.file_utils:removing temp file /tmp/tmpgucbs7jt
INFO:pytorch_pretrained_bert.modeling:loading archive file https://s3.amazonaws.com/models.huggingface.co/bert/bert-base-cased.tar.gz from cache at cache/a803ce83ca27fecf74c355673c434e51c265fb8a3e0e57ac62a80e38ba98d384.681017f415dfb33ec8d0e04fe51a619f3f01532ecea04edbfd48c5d

In [0]:
model.to(device)

BertForSequenceClassification(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(28996, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): BertLayerNorm()
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0): BertLayer(
          (attention): BertAttention(
            (self): BertSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): BertLayerNorm()
              (dropout): Dropout(p=0.1, inplace=False)
   

In [0]:
param_optimizer = list(model.named_parameters())
no_decay = ['bias', 'LayerNorm.bias', 'LayerNorm.weight']
optimizer_grouped_parameters = [
    {'params': [p for n, p in param_optimizer if not any(nd in n for nd in no_decay)], 'weight_decay': 0.01},
    {'params': [p for n, p in param_optimizer if any(nd in n for nd in no_decay)], 'weight_decay': 0.0}
    ]

In [0]:
optimizer = BertAdam(optimizer_grouped_parameters,
                     lr=LEARNING_RATE,
                     warmup=WARMUP_PROPORTION,
                     t_total=num_train_optimization_steps)

In [0]:
global_step = 0
nb_tr_steps = 0
tr_loss = 0

In [0]:
logger.info("***** Running training *****")
logger.info("  Num examples = %d", train_examples_len)
logger.info("  Batch size = %d", TRAIN_BATCH_SIZE)
logger.info("  Num steps = %d", num_train_optimization_steps)
all_input_ids = torch.tensor([f.input_ids for f in train_features], dtype=torch.long)
all_input_mask = torch.tensor([f.input_mask for f in train_features], dtype=torch.long)
all_segment_ids = torch.tensor([f.segment_ids for f in train_features], dtype=torch.long)

if OUTPUT_MODE == "classification":
    all_label_ids = torch.tensor([f.label_id for f in train_features], dtype=torch.long)
elif OUTPUT_MODE == "regression":
    all_label_ids = torch.tensor([f.label_id for f in train_features], dtype=torch.float)

INFO:root:***** Running training *****
INFO:root:  Num examples = 31604
INFO:root:  Batch size = 24
INFO:root:  Num steps = 3948


In [0]:
train_data = TensorDataset(all_input_ids, all_input_mask, all_segment_ids, all_label_ids)
train_sampler = RandomSampler(train_data)
train_dataloader = DataLoader(train_data, sampler=train_sampler, batch_size=TRAIN_BATCH_SIZE)

In [0]:
### Set training eposchs again
NUM_TRAIN_EPOCHS = 3

In [0]:
%%time
model.train()
for _ in trange(int(NUM_TRAIN_EPOCHS), desc="Epoch"):
    tr_loss = 0
    nb_tr_examples, nb_tr_steps = 0, 0
    for step, batch in enumerate(tqdm_notebook(train_dataloader, desc="Iteration")):
        batch = tuple(t.to(device) for t in batch)
        input_ids, input_mask, segment_ids, label_ids = batch

        logits = model(input_ids, segment_ids, input_mask, labels=None)

        if OUTPUT_MODE == "classification":
            loss_fct = CrossEntropyLoss()
            loss = loss_fct(logits.view(-1, num_labels), label_ids.view(-1))
        elif OUTPUT_MODE == "regression":
            loss_fct = MSELoss()
            loss = loss_fct(logits.view(-1), label_ids.view(-1))

        if GRADIENT_ACCUMULATION_STEPS > 1:
            loss = loss / GRADIENT_ACCUMULATION_STEPS

        loss.backward()
        print("\r%f" % loss, end='')
        
        tr_loss += loss.item()
        nb_tr_examples += input_ids.size(0)
        nb_tr_steps += 1
        if (step + 1) % GRADIENT_ACCUMULATION_STEPS == 0:
            optimizer.step()
            optimizer.zero_grad()
            global_step += 1


Epoch:   0%|          | 0/3 [00:00<?, ?it/s]

0.276565

Epoch:  33%|███▎      | 1/3 [25:19<50:38, 1519.14s/it]

0.155372


0.373898

...

0.036843


0.075349

0.058139

Epoch: 100%|██████████| 3/3 [1:15:57<00:00, 1519.28s/it]

0.032925
CPU times: user 47min 31s, sys: 28min 2s, total: 1h 15min 33s
Wall time: 1h 15min 57s


In [0]:
model_to_save = model.module if hasattr(model, 'module') else model  # Only save the model it-self

# If we save using the predefined names, we can load using `from_pretrained`
output_model_file = os.path.join(OUTPUT_DIR, WEIGHTS_NAME)
output_config_file = os.path.join(OUTPUT_DIR, CONFIG_NAME)

torch.save(model_to_save.state_dict(), output_model_file)
model_to_save.config.to_json_file(output_config_file)
tokenizer.save_vocabulary(OUTPUT_DIR)

'outputs/hate_speech/vocab.txt'

In [0]:
# need to split the train and test first, before actually moving further. Lets do that for next run, right now the model is getting trained on whole train

#### Evaluation

Go into the outputs/hate_speech directory where the fine tuned models will be saved. There, you should find 3 files; config.json, pytorch_model.bin, vocab.txt.
Archive the two files (I use 7zip for archiving) config.json, and pytorch_model.bin into a .tar file.
Compress the .tar file into gzip format. Now the file should be something like hate_speech.tar.gz
Copy the compressed file into the cache/ directory.[link text](https://)

In [0]:
# !mkdir /content/outputs/hate_speech/archive

In [0]:
# !cp /content/outputs/hate_speech/config.json /content/outputs/hate_speech/archive
# !cp /content/outputs/hate_speech/pytorch_model.bin /content/outputs/hate_speech/archive

In [0]:
%cd /content/outputs/hate_speech

/content/outputs/hate_speech


In [0]:
# !tar -zcvf hate_speech.tar.gz /content/outputs/hate_speech/config.json /content/outputs/hate_speech/pytorch_model.bin

!tar -czvf hate_speech.tar.gz config.json pytorch_model.bin

config.json
pytorch_model.bin


In [0]:
!cp  hate_speech.tar.gz /content/cache

In [0]:
%cd /content

/content


In [0]:
import torch
import numpy as np
import pickle

from sklearn.metrics import matthews_corrcoef, confusion_matrix

from torch.utils.data import (DataLoader, RandomSampler, SequentialSampler,
                              TensorDataset)
from torch.utils.data.distributed import DistributedSampler
from torch.nn import CrossEntropyLoss, MSELoss

# from tools import *
from multiprocessing import Pool, cpu_count
# import convert_examples_to_features

from tqdm import tqdm_notebook, trange
import os
from pytorch_pretrained_bert import BertTokenizer, BertModel, BertForMaskedLM, BertForSequenceClassification
from pytorch_pretrained_bert.optimization import BertAdam, WarmupLinearSchedule

# OPTIONAL: if you want to have more information on what's happening, activate the logger as follows
import logging
logging.basicConfig(level=logging.INFO)

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [0]:
# The input data dir. Should contain the .tsv files (or other data files) for the task.
DATA_DIR = ""

# Bert pre-trained model selected in the list: bert-base-uncased, 
# bert-large-uncased, bert-base-cased, bert-large-cased, bert-base-multilingual-uncased,
# bert-base-multilingual-cased, bert-base-chinese.
BERT_MODEL = 'hate_speech.tar.gz'

# The name of the task to train.I'm going to name this 'yelp'.
TASK_NAME = 'hate_speech'

# The output directory where the fine-tuned model and checkpoints will be written.
OUTPUT_DIR = f'outputs/{TASK_NAME}/'

# The directory where the evaluation reports will be written to.
REPORTS_DIR = f'reports/{TASK_NAME}_evaluation_reports/'

# This is where BERT will look for pre-trained models to load parameters from.
CACHE_DIR = 'cache/'

# The maximum total input sequence length after WordPiece tokenization.
# Sequences longer than this will be truncated, and sequences shorter than this will be padded.
MAX_SEQ_LENGTH = 128

TRAIN_BATCH_SIZE = 24
EVAL_BATCH_SIZE = 8
LEARNING_RATE = 2e-5
NUM_TRAIN_EPOCHS = 3
RANDOM_SEED = 42
GRADIENT_ACCUMULATION_STEPS = 1
WARMUP_PROPORTION = 0.1
OUTPUT_MODE = 'classification'

CONFIG_NAME = "config.json"
WEIGHTS_NAME = "pytorch_model.bin"

In [0]:
if os.path.exists(REPORTS_DIR) and os.listdir(REPORTS_DIR):
        REPORTS_DIR += f'/report_{len(os.listdir(REPORTS_DIR))}'
        os.makedirs(REPORTS_DIR)
if not os.path.exists(REPORTS_DIR):
    os.makedirs(REPORTS_DIR)
    REPORTS_DIR += f'/report_{len(os.listdir(REPORTS_DIR))}'
    os.makedirs(REPORTS_DIR)

def get_eval_report(task_name, labels, preds):
    mcc = matthews_corrcoef(labels, preds)
    tn, fp, fn, tp = confusion_matrix(labels, preds).ravel()
    return {
        "task": task_name,
        "mcc": mcc,
        "tp": tp,
        "tn": tn,
        "fp": fp,
        "fn": fn
    }

def compute_metrics(task_name, labels, preds):
    assert len(preds) == len(labels)
    return get_eval_report(task_name, labels, preds)

In [0]:
# Load pre-trained model tokenizer (vocabulary)
tokenizer = BertTokenizer.from_pretrained(OUTPUT_DIR + 'vocab.txt', do_lower_case=False)

processor = BinaryClassificationProcessor()

eval_examples = processor.get_dev_examples(DATA_DIR)
label_list = processor.get_labels() # [0, 1] for binary classification
num_labels = len(label_list)
eval_examples_len = len(eval_examples)

label_map = {label: i for i, label in enumerate(label_list)}
eval_examples_for_processing = [(example, label_map, MAX_SEQ_LENGTH, tokenizer, OUTPUT_MODE) for example in eval_examples]

process_count = cpu_count() - 1
if __name__ ==  '__main__':
    print(f'Preparing to convert {eval_examples_len} examples..')
    print(f'Spawning {process_count} processes..')
    with Pool(process_count) as p:
        eval_features = list(tqdm_notebook(p.imap(convert_example_to_feature, eval_examples_for_processing), total=eval_examples_len))

INFO:pytorch_pretrained_bert.tokenization:loading vocabulary file outputs/hate_speech/vocab.txt


Preparing to convert 7902 examples..
Spawning 1 processes..


In [0]:
all_input_ids = torch.tensor([f.input_ids for f in eval_features], dtype=torch.long)
all_input_mask = torch.tensor([f.input_mask for f in eval_features], dtype=torch.long)
all_segment_ids = torch.tensor([f.segment_ids for f in eval_features], dtype=torch.long)

# In [25]:
if OUTPUT_MODE == "classification":
    all_label_ids = torch.tensor([f.label_id for f in eval_features], dtype=torch.long)
elif OUTPUT_MODE == "regression":
    all_label_ids = torch.tensor([f.label_id for f in eval_features], dtype=torch.float)

# In [26]:
eval_data = TensorDataset(all_input_ids, all_input_mask, all_segment_ids, all_label_ids)

# Run prediction for full data
eval_sampler = SequentialSampler(eval_data)
eval_dataloader = DataLoader(eval_data, sampler=eval_sampler, batch_size=EVAL_BATCH_SIZE)

In [0]:
# In [29]:
# Load pre-trained model (weights)
model = BertForSequenceClassification.from_pretrained(CACHE_DIR + BERT_MODEL, cache_dir=CACHE_DIR, num_labels=len(label_list))

INFO:pytorch_pretrained_bert.modeling:loading archive file cache/hate_speech.tar.gz
INFO:pytorch_pretrained_bert.modeling:extracting archive file cache/hate_speech.tar.gz to temp dir /tmp/tmpd_iond1_
INFO:pytorch_pretrained_bert.modeling:Model config {
  "attention_probs_dropout_prob": 0.1,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "initializer_range": 0.02,
  "intermediate_size": 3072,
  "max_position_embeddings": 512,
  "num_attention_heads": 12,
  "num_hidden_layers": 12,
  "type_vocab_size": 2,
  "vocab_size": 28996
}



Okay so main problem here is, the tar.gz file isnt only containing the config and bin file, its kind layered structure and hence its not able to extract it properly, We need to upload it manually it seems

In [0]:
model.to(device)

BertForSequenceClassification(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(28996, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): BertLayerNorm()
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0): BertLayer(
          (attention): BertAttention(
            (self): BertSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): BertLayerNorm()
              (dropout): Dropout(p=0.1, inplace=False)
   

In [0]:
%%time
model.eval()
eval_loss = 0
nb_eval_steps = 0
preds = []

for input_ids, input_mask, segment_ids, label_ids in tqdm_notebook(eval_dataloader, desc="Evaluating"):
    input_ids = input_ids.to(device)
    input_mask = input_mask.to(device)
    segment_ids = segment_ids.to(device)
    label_ids = label_ids.to(device)

    with torch.no_grad():
        logits = model(input_ids, segment_ids, input_mask, labels=None)

    # create eval loss and other metric required by the task
    if OUTPUT_MODE == "classification":
        loss_fct = CrossEntropyLoss()
        tmp_eval_loss = loss_fct(logits.view(-1, num_labels), label_ids.view(-1))
    elif OUTPUT_MODE == "regression":
        loss_fct = MSELoss()
        tmp_eval_loss = loss_fct(logits.view(-1), label_ids.view(-1))

    eval_loss += tmp_eval_loss.mean().item()
    nb_eval_steps += 1
    if len(preds) == 0:
        preds.append(logits.detach().cpu().numpy())
    else:
        preds[0] = np.append(
            preds[0], logits.detach().cpu().numpy(), axis=0)

eval_loss = eval_loss / nb_eval_steps
preds = preds[0]
if OUTPUT_MODE == "classification":
    preds = np.argmax(preds, axis=1)
elif OUTPUT_MODE == "regression":
    preds = np.squeeze(preds)
result = compute_metrics(TASK_NAME, all_label_ids.numpy(), preds)

result['eval_loss'] = eval_loss

output_eval_file = os.path.join(REPORTS_DIR, "eval_results.txt")
with open(output_eval_file, "w") as writer:
    logger.info("***** Eval results *****")
    for key in (result.keys()):
        logger.info("  %s = %s", key, str(result[key]))
        writer.write("%s = %s\n" % (key, str(result[key])))

INFO:root:***** Eval results *****
INFO:root:  task = hate_speech
INFO:root:  mcc = 0.824360726779699
INFO:root:  tp = 4574
INFO:root:  tn = 2672
INFO:root:  fp = 260
INFO:root:  fn = 396
INFO:root:  eval_loss = 0.261138613467757



CPU times: user 1min 26s, sys: 40.7 s, total: 2min 7s
Wall time: 2min 7s


In [0]:
pd.read_csv('/content/reports/hate_speech_evaluation_reports/report_0/eval_results.txt')

,task = hate_speech
0,mcc = 0.824360726779699
1,tp = 4574
2,tn = 2672
3,fp = 260
4,fn = 396
5,eval_loss = 0.261138613467757


In [0]:
# result with 3 epcohs
tp = 4574
tn = 2672
fp = 260
fn = 396
eval_loss = 0.261138613467757

In [0]:
# results on one epoch
# tp = 4518
# tn = 2645
# fp = 287
# fn = 452
# eval_loss =  0.22225678609570992

In [0]:
(tp+tn)/(tp+tn+fp+fn)
## accuracy

0.9169830422677803

In [0]:
2*tp/(2*tp + fp + fn)
### f1 score

0.9330885352917176

In [0]:
# epochs = 1
# 0.9064793723108074 -  Accuracy
# 0.9243989769820972 -  F1 Score

# epochs = 3
# 0.9169830422677803 - Accuracy
# 0.9330885352917176 -  F1 Score

In [0]:
### great it really is SOTA isnt it!!!! 3% improvement in F1-Score, and 4% in accuracy as well,
### with just one epoch of training/

In [0]:
### lets save these files offline first

In [0]:
!tar -czvf outputs.tar.gz outputs

outputs/
outputs/hate_speech/
outputs/hate_speech/vocab.txt
outputs/hate_speech/hate_speech.tar.gz
outputs/hate_speech/pytorch_model.bin
outputs/hate_speech/reports/
outputs/hate_speech/reports/hate_speech_evaluation_reports/
outputs/hate_speech/reports/hate_speech_evaluation_reports/report_0/
outputs/hate_speech/config.json


In [0]:
!tar -czvf reports.tar.gz reports

reports/
reports/hate_speech_evaluation_reports/
reports/hate_speech_evaluation_reports/report_0/
reports/hate_speech_evaluation_reports/report_0/eval_results.txt
reports/hate_speech_evaluation_report/
reports/hate_speech_evaluation_report/report_0/


In [0]:
!tar -czvf cache.tar.gz cache

cache/
cache/hate_speech.tar.gz
cache/a803ce83ca27fecf74c355673c434e51c265fb8a3e0e57ac62a80e38ba98d384.681017f415dfb33ec8d0e04fe51a619f3f01532ecea04edbfd48c5d160550d9c
cache/a803ce83ca27fecf74c355673c434e51c265fb8a3e0e57ac62a80e38ba98d384.681017f415dfb33ec8d0e04fe51a619f3f01532ecea04edbfd48c5d160550d9c.json


#### Single sentence evaluation

In [0]:
!pip install pytorch-pretrained-bert

     |████████████████████████████████| 133kB 9.4MB/s 
     |████████████████████████████████| 645kB 32.2MB/s 


In [0]:
import torch
import numpy as np
import pickle

from sklearn.metrics import matthews_corrcoef, confusion_matrix

from torch.utils.data import (DataLoader, RandomSampler, SequentialSampler,
                              TensorDataset)
from torch.utils.data.distributed import DistributedSampler
from torch.nn import CrossEntropyLoss, MSELoss

# from tools import *
from multiprocessing import Pool, cpu_count
# import convert_examples_to_features

from tqdm import tqdm_notebook, trange
import os
from pytorch_pretrained_bert import BertTokenizer, BertModel, BertForMaskedLM, BertForSequenceClassification
from pytorch_pretrained_bert.optimization import BertAdam, WarmupLinearSchedule

# OPTIONAL: if you want to have more information on what's happening, activate the logger as follows
import logging
logging.basicConfig(level=logging.INFO)

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# The input data dir. Should contain the .tsv files (or other data files) for the task.
DATA_DIR = ""

# Bert pre-trained model selected in the list: bert-base-uncased, 
# bert-large-uncased, bert-base-cased, bert-large-cased, bert-base-multilingual-uncased,
# bert-base-multilingual-cased, bert-base-chinese.
BERT_MODEL = 'hate_speech.tar.gz'

# The name of the task to train.I'm going to name this 'yelp'.
TASK_NAME = 'hate_speech'

# The output directory where the fine-tuned model and checkpoints will be written.
OUTPUT_DIR = f'outputs/{TASK_NAME}/'

# The directory where the evaluation reports will be written to.
REPORTS_DIR = f'reports/{TASK_NAME}_evaluation_reports/'

# This is where BERT will look for pre-trained models to load parameters from.
CACHE_DIR = 'cache/'

# The maximum total input sequence length after WordPiece tokenization.
# Sequences longer than this will be truncated, and sequences shorter than this will be padded.
MAX_SEQ_LENGTH = 128

TRAIN_BATCH_SIZE = 24
EVAL_BATCH_SIZE = 8
LEARNING_RATE = 2e-5
NUM_TRAIN_EPOCHS = 3
RANDOM_SEED = 42
GRADIENT_ACCUMULATION_STEPS = 1
WARMUP_PROPORTION = 0.1
OUTPUT_MODE = 'classification'

CONFIG_NAME = "config.json"
WEIGHTS_NAME = "pytorch_model.bin"

if os.path.exists(REPORTS_DIR) and os.listdir(REPORTS_DIR):
        REPORTS_DIR += f'/report_{len(os.listdir(REPORTS_DIR))}'
        os.makedirs(REPORTS_DIR)
if not os.path.exists(REPORTS_DIR):
    os.makedirs(REPORTS_DIR)
    REPORTS_DIR += f'/report_{len(os.listdir(REPORTS_DIR))}'
    os.makedirs(REPORTS_DIR)

def get_eval_report(task_name, labels, preds):
    mcc = matthews_corrcoef(labels, preds)
    tn, fp, fn, tp = confusion_matrix(labels, preds).ravel()
    return {
        "task": task_name,
        "mcc": mcc,
        "tp": tp,
        "tn": tn,
        "fp": fp,
        "fn": fn
    }

def compute_metrics(task_name, labels, preds):
    assert len(preds) == len(labels)
    return get_eval_report(task_name, labels, preds)

In [0]:
# Load pre-trained model tokenizer (vocabulary)
tokenizer = BertTokenizer.from_pretrained(OUTPUT_DIR + 'vocab.txt', do_lower_case=False)

ERROR:pytorch_pretrained_bert.tokenization:Model name 'outputs/hate_speech/vocab.txt' was not found in model name list (bert-base-uncased, bert-large-uncased, bert-base-cased, bert-large-cased, bert-base-multilingual-uncased, bert-base-multilingual-cased, bert-base-chinese). We assumed 'outputs/hate_speech/vocab.txt' was a path or url but couldn't find any file associated to this path or url.


In [0]:

processor = BinaryClassificationProcessor()
# eval_examples = processor.get_dev_examples(DATA_DIR)
## MANUALLY SETTING TEST SENTENCE HERE ##

eval_examples = [InputExample(guid=0, text_a='white', text_b=None, label='1')]
####

In [0]:


label_list = processor.get_labels() # [0, 1] for binary classification
num_labels = len(label_list)
eval_examples_len = len(eval_examples)

label_map = {label: i for i, label in enumerate(label_list)}
eval_examples_for_processing = [(example, label_map, MAX_SEQ_LENGTH, tokenizer, OUTPUT_MODE) for example in eval_examples]


process_count = cpu_count() - 1
if __name__ ==  '__main__':
    print(f'Preparing to convert {eval_examples_len} examples..')
    print(f'Spawning {process_count} processes..')
    with Pool(process_count) as p:
        eval_features = list(tqdm_notebook(p.imap(convert_example_to_feature, eval_examples_for_processing), total=eval_examples_len))

Preparing to convert 1 examples..
Spawning 1 processes..


In [0]:
all_input_ids = torch.tensor([f.input_ids for f in eval_features], dtype=torch.long)
all_input_mask = torch.tensor([f.input_mask for f in eval_features], dtype=torch.long)
all_segment_ids = torch.tensor([f.segment_ids for f in eval_features], dtype=torch.long)

# In [25]:
if OUTPUT_MODE == "classification":
    all_label_ids = torch.tensor([f.label_id for f in eval_features], dtype=torch.long)
elif OUTPUT_MODE == "regression":
    all_label_ids = torch.tensor([f.label_id for f in eval_features], dtype=torch.float)

# In [26]:
eval_data = TensorDataset(all_input_ids, all_input_mask, all_segment_ids, all_label_ids)

# Run prediction for full data
eval_sampler = SequentialSampler(eval_data)
eval_dataloader = DataLoader(eval_data, sampler=eval_sampler, batch_size=EVAL_BATCH_SIZE)

In [0]:
# Load pre-trained model (weights)
model = BertForSequenceClassification.from_pretrained(CACHE_DIR + BERT_MODEL, cache_dir=CACHE_DIR, num_labels=len(label_list))

INFO:pytorch_pretrained_bert.modeling:loading archive file cache/hate_speech.tar.gz
INFO:pytorch_pretrained_bert.modeling:extracting archive file cache/hate_speech.tar.gz to temp dir /tmp/tmp51wd13_f
INFO:pytorch_pretrained_bert.modeling:Model config {
  "attention_probs_dropout_prob": 0.1,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "initializer_range": 0.02,
  "intermediate_size": 3072,
  "max_position_embeddings": 512,
  "num_attention_heads": 12,
  "num_hidden_layers": 12,
  "type_vocab_size": 2,
  "vocab_size": 28996
}



In [0]:
model.to(device)

BertForSequenceClassification(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(28996, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): BertLayerNorm()
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0): BertLayer(
          (attention): BertAttention(
            (self): BertSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): BertLayerNorm()
              (dropout): Dropout(p=0.1, inplace=False)
   

In [0]:
model.eval()
eval_loss = 0
nb_eval_steps = 0
preds = []

for input_ids, input_mask, segment_ids, label_ids in tqdm_notebook(eval_dataloader, desc="Evaluating"):
    input_ids = input_ids.to(device)
    input_mask = input_mask.to(device)
    segment_ids = segment_ids.to(device)
    label_ids = label_ids.to(device)

    with torch.no_grad():
        logits = model(input_ids, segment_ids, input_mask, labels=None)

    # create eval loss and other metric required by the task
    if OUTPUT_MODE == "classification":
        loss_fct = CrossEntropyLoss()
        tmp_eval_loss = loss_fct(logits.view(-1, num_labels), label_ids.view(-1))
    elif OUTPUT_MODE == "regression":
        loss_fct = MSELoss()
        tmp_eval_loss = loss_fct(logits.view(-1), label_ids.view(-1))

    eval_loss += tmp_eval_loss.mean().item()
    nb_eval_steps += 1
    if len(preds) == 0:
        preds.append(logits.detach().cpu().numpy())
    else:
        preds[0] = np.append(
            preds[0], logits.detach().cpu().numpy(), axis=0)

eval_loss = eval_loss / nb_eval_steps
preds = preds[0]

In [0]:
from scipy.special import softmax

pred_probs = softmax(preds, axis=1)

In [0]:
pred_probs

array([[0.9905739 , 0.00942601]], dtype=float32)

In [0]:
preds.shape

(1, 2)

In [0]:
np.argmax(pred_probs)

0

In [0]:
np.max(pred_probs)

0.9905739